In [24]:
import soundfile
import sounddevice
import codecs
import glob
import multiprocessing
import re
import nltk
import gensim.models.word2vec as w2v
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gensim
from sklearn.model_selection import train_test_split

In [25]:
import keras
smodel = keras.models.load_model("./DeadSimpleSpeechRecognizer/chippy_v1.model")
model = keras.models.load_model("model1.chatbot")

In [26]:
w2vmodel="chat_model1-300.w2v"   #filename
num_features = 300    #number of features in the model
maxsent = 10          #maximum words in a sentence.
intent=[]

In [27]:
# Second dimension of the feature is dim2
feature_dim_2 = 11

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 2

In [28]:
tex2vec =  gensim.models.KeyedVectors.load(w2vmodel)

In [29]:
interrupted = False
yesno_state = False
listen_state = False
talk_state = False
cmd = ""
headless = False
i_quit = False

### GET INTENT

In [30]:
import glob
import fileinput
fls = glob.glob('./data/*/input.txt')
intent=[]
prep_x = []
prep_y = []
for i in fls:
    for tx in fileinput.input(i):
        intent.append(i.split('\\')[-2])
fileinput.close()

intent = list(set(intent))
intent.sort()
print(intent)

['affirmasi', 'cctv_lewatmana', 'goodbye', 'lihat_gambar', 'negasi', 'salam']


In [31]:
#loader all intent
for i in intent:
    exect = 'from data.'+i+" import "+i
    print(exect)
    exec(exect)

from data.affirmasi import affirmasi
from data.cctv_lewatmana import cctv_lewatmana
from data.goodbye import goodbye
from data.lihat_gambar import lihat_gambar
from data.negasi import negasi
from data.salam import salam


### PREDICTION ROUTINE

In [32]:
from preprocess import *

min_pred = {'lihat_gambar': 0.2 ,
            'cctv_lewatmana': 0.2, 
            'salam': 0.1, 
            'affirmasi': 0.1,
            'negasi': 0.1,
            'goodbye': 0.1}

def pred(sentence):
    predictions=model.predict(np.array([swv_ar(twl(sentence))]))
    cls_pred = np.argmax(predictions,axis=1)
   
    result={'intent': intent[cls_pred[0]], 'score': predictions.max(), 'sentence': sentence}
    return result

intention = 'None'
followup = 'None'
intentfu = 'None'


def mainchat(sentence):
    global intention, followup, intentfu
    result = pred(sentence)

    #PERUBAHAN INTENT APABILA SCORE DIBAWA YANG DI TENTUKAN
    if result['score'] > min_pred[result['intent']]:
        #print('i do intent:', result['intent'])
        pass
    else:
        #print('i change to noting')
        result['intent']='chitchat'

    #EXEKUSI FOLLOWUP SECARA PRIORITAS
    if followup != 'None':
            if result['intent'] == 'negasi':
                #print("pembatalan")
                rslt={'intent': 'None', 'followup': 'None', 'name': 'None', 'prompt': 'batal' }
            else:
                #print("a followup:", result)
                instruction = intentfu+'.'+"input(\'"+result['sentence']+"\')"
                intentfu = result['intent']
                #print(instruction)
                rslt=eval(instruction)
    else:
        #print('not followup', result)
        if result['intent'] != 'chitchat':
            instruction = result['intent']+'.'+"input(\'"+result['sentence']+"\')"
            #print(instruction)
            rslt=eval(instruction)
        else:
            
            if result['intent'] == 'goodbye':
                print("I will quit")
                i_quit=True
            #TARUH DISINI UNTUK RUTIN CHITCHAT - dan kurang mengerti apa yang di bilang.
            instruction={'name': 'None'}
            rslt={'name': 'None', 'followup': 'None', 'prompt': 'ada yang saya bisa bantu?'}
        #print(instruction, "intent driven")
        
    #print(result)
    if type(rslt) is not dict:
        intention = 'None'
        followup = 'None'
    else:
        intention = result['intent']
        followup = rslt['name']
        intentfu = rslt['followup']
    return rslt

#[tex2vec.most_similar([predictions[0][i]])[0] for i in range(maxsent)]

def process_reply(reply):
    name=reply['name']
    prompt=reply['prompt']
    followup=reply['followup']
    print('Bot say:',prompt)
    
# Predicts one sample
def predict(filepath, model):
    global predicting
    predicting = True
    sv,sr = soundfile.read('./Ring09.wav')
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    y_pred = model.predict(sample_reshaped)
    y_max = np.max(y_pred)
    ypred = np.argmax(y_pred)
    if y_max>0.9 and ypred==1:
        followup = False
        print("Predicted:" ,get_labels()[0][ypred],y_max, ypred)
        sounddevice.play(sv,samplerate=16000)
        #sudah di kenali 
        
        speech = get_speech()
        reply = mainchat(speech)
        talk(reply['prompt'])
        print(reply['followup'])
        
        if reply['followup'] != 'None':
            followup=True
            print('followup teridentifikasi')
        
        while followup:
            time.sleep(1)
            playding()
            speech = get_speech()
            reply = mainchat(speech)
            talk(reply['prompt'])
            if reply['followup'] != 'None':
                followup=True
                predicting=False
            else:
                followup=False
                predicting=False
                break
    predicting=False
    return get_labels()[0][ypred]



In [33]:
def remove_unchar(flist):
    try:
        flist.remove("?")
    except:
        pass

def twl(sentence, rempunct=True, flat=True):
    """Tokenize word dari sebuah sentence/kalimat"""
    if not flat:
        sntoken = nltk.sent_tokenize(sentence)
    else:
        sntoken = [sentence]
    for i in range(len(sntoken)):
        tokens = nltk.word_tokenize(sntoken[i])
        if rempunct==True:
            type(tokens)
            text = nltk.Text(tokens)
            type(text)  
            sntoken[i] = [w.lower() for w in text if w.isalpha()]
        else: sntoken[i] = tokens
    if len(sntoken)==1: 
        sntoken = sntoken[0]
        remove_unchar(sntoken)
    for i in sntoken:
        remove_unchar(i)
    if sntoken==[]: sntoken=['yang']
    #print(sntoken)
    return sntoken

def zerolistmaker(n):
    """Membuat list berisi 0, mirip seperti np.zeros"""
    listofzeros = [1] * n
    return listofzeros



def swv_ar(sentence, maxword=maxsent, vecsize=num_features, frontpad=True):
    """Sentence word2vec to array matrices for processing"""
    senarray=[]
    if len(sentence)>=maxsent: 
        print("Sentence overflow:",sentence)
        sentence=sentence[0:maxsent-1]
    if type(sentence[0])==list:
        for i in range(len(sentence)):
            for k in range(len(sentence[i])):
                #print(tex2vec[sentence[i][k]])
                try:
                    senarray.append(tex2vec[sentence[i][k]])
                except Exception as e:
                    print("Problem at phrase:", sentence[i][k])
                    
    else:
        for i in range(len(sentence)):
            try:
                senarray.append(tex2vec[sentence[i]])
            except Exception as e:
                print("Problem at phrase:", sentence[i])
                
    zr=zerolistmaker(num_features)
    #reverse if want to add a padding in front
    if frontpad: senarray.reverse()

    #add the padding
    for i in range(maxword-len(senarray)):     
        senarray.append(zr)

    #reverse again
    if frontpad: senarray.reverse()            
    
    return np.array(senarray)

def wv_ts(array, weight=False):
    """Wave2vec matrix to sentence - weight untuk menampilkan bobot"""
    result=[]
    for i in array:
        try:
            word=tex2vec.similar_by_vector(vector=i,topn=1)
            if word[0][1] == 0: word[0]="."
                
            #bikin yang mau di append ada bobotnya gak?
            if weight == False:
                to_app = word[0][0]
            else: to_app = word[0]
                
            result.append(to_app)
        except:
            pass
    return result

In [34]:
import soundfile
import itertools
import time
import librosa
import sounddevice

DURATION=20
gain=10
high=2000
low=100
screenwidth=79
predicting=False

def record(length=1, reclength=1, filename=None, thres=0):
    """ 
    Merekam suara secara stream dan metode callback
    """

    global cumulated_status, end_count, start_count, recording, magnitudo, audiodata, predicting, i_quit
    predicting=False
    end_count=False
    start_count = 0
    recording=False
    magnitudo=[]
    audiodata=[]
    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / screenwidth
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()

        def callback(indata, frames, time, status):
            global cumulated_status, audiodata, magnitudo, end_count, start_count, recording, smodel, predicting, i_quit
            
            cumulated_status |= status
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize

                rms = librosa.feature.rmse(S=indata)
                rms = int(rms*32768)
                start_count += 1
                if rms>=thres:
                    if not recording :                    #and not end_count
                        #print("Start record")
                        recording = True
                        start_count = 0
                        
                        
                if recording:
                    audiodata.extend(itertools.chain(indata.tolist()))
                    magnitudo.append(magnitude)
                    if start_count == int(samplerate / (samplerate * DURATION / 1000)):
                        #print("End record")
                        start_count=0
                        end_count=True
                        recording=False
                        try:
                            if not predicting:
                                print("Predict")
                                soundfile.write("temp.wav",audiodata,16000)
                                predict("temp.wav", model=smodel)
                            pass
                        except:
                            pass
                        audiodata=[]



        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            while True:
                #response = input()
                #if response in ('', 'q', 'Q'):
                time.sleep(length)
                break
            if filename!=None: soundfile.write(filename,audiodata,16000)

        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)

In [35]:
print('intention:',intention)
print('followup:',followup)
print('intentfu:',intentfu)
#myinput = input('>')
#reply = mainchat(myinput)
#print(reply)
#process_reply(reply)
print('intention:',intention)
print('followup:',followup)
print('intentfu:',intentfu)

intention: None
followup: None
intentfu: None
intention: None
followup: None
intentfu: None


In [36]:
import nltk
reply = mainchat("Hallo apa kabar")
print(reply['prompt'])

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


halo


In [37]:
predicting = False

### SELENIUM ROUTINE

In [38]:
def cek_talk_or_speak():
    try:
        m = s_iface.find_element_by_class_name("goog-toolbar-button-checked")
        return True
    except:
        return False

def get_speech():
    if not cek_talk_or_speak():
        record = s_iface.find_element_by_xpath('//*[@id="gt-speech"]/span')
        record.click()
        time.sleep(4)
        #record
        record.click()
        source = s_iface.find_element_by_xpath('//*[@id="source"]')
        a = source.get_attribute("value")
        clear()
        playding()
    return a

def clear():
    source = s_iface.find_element_by_xpath('//*[@id="source"]')
    source.clear()
    
def talk(speech):
    if not cek_talk_or_speak():
        tts = s_iface.find_element_by_xpath('//*[@id="source"]')
        tts.clear()
        time.sleep(0.5)
        tts.send_keys(str(speech))
        time.sleep(1)

        talk = s_iface.find_element_by_id('gt-res-listen') 
        talk.click()
        time.sleep(1)

def check_live():
    try:
        result=s_iface.execute_script("return document.readyState;")
        return True
    except:
        return False

def playding():
    wv,sr = soundfile.read("./ehm.wav")
    sounddevice.play(wv,samplerate=44100)

### SELENIUM LOAD

In [39]:
from selenium import webdriver


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--use-fake-ui-for-media-stream")
if headless:
    chrome_options.add_argument("--headless")  

s_iface = webdriver.Chrome(chrome_options=chrome_options)
s_iface.get('https://translate.google.com/?#id/id')
assert "Google Translate" in s_iface.title

### SPEECH LOOP

In [ ]:
while True:
    talky = input(">")
    if talky == 'q': break
    talk(talky)

In [40]:
predict("temp.wav", model = smodel)

Predicted: cipi 0.9999567 1


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


None


'cipi'

In [41]:
record(length=125,filename="record.wav",thres=1100)
print("\n\nEnd Recording")

Predict
Predicted: cipi 0.99996805 1


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


None
Predict
Predict
Predicted: cipi 0.9983246 1
None
Predict
Predict
Predict
Predict
Predicted: cipi 0.9999095 1
lihat_gambar
followup teridentifikasi
Predict
Predict
Predict
Predicted: cipi 0.9999373 1
lihat_gambar
followup teridentifikasi
memprosess gambar: kuda di kebun tanggal 7
name 'logging' is not defined


End Recording
